### Packages

In [32]:
# import packages
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import scipy.io
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from Optimization import *
from opt_utils_v1a import load_params_and_grads, initialize_parameters, forward_propagation, backward_propagation
from opt_utils_v1a import compute_cost, predict, predict_dec, plot_decision_boundary, load_dataset
from copy import deepcopy

### Dataset

#### Use the a model and datasets (a complex classification problem that you can choose on your own) and test different regularization techniques:

Here we will be using the same dataset as the assignment 2 in which the problem is to classify a set of images as cats or not. 
Although it is not a requirement from this assignment, we will first try to solve this problem without regularization whatsoever.

To follow the same idea from the assignment 2, for the training of the model we will be using the given dataset by the teacher as a file named train_catvnoncat.h5. While the first 100 images from the sets of cats and dogs labeled as "train" retrieved from https://www.kaggle.com/datasets/samuelcortinhas/cats-and-dogs-image-classification?select=train will be used for our testing. And the next 100 images from those that same sets will be used for our tuning set. We will first process them to reduce their pixel resolution to the same as the ones we used in the training (64 pixels x 64 pixels) and then represent them as arrays.
##### 0.  Preprocess datasets

In [10]:
# Some useful functions and packages for processing and loading the datasets:


def load_dataset():
    train_dataset = h5py.File('datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

def preprocess_dataset(dataset):
    dataset_flatten = dataset.reshape(dataset.shape[0],-1).T
    return dataset_flatten/255 

train_set_x = preprocess_dataset(train_set_x_orig) 
test_set_x = preprocess_dataset(test_set_x_orig)

## Extract our own images from cats and dogs
import os

def process_images(directory, target_size=(64, 64), image_range=range(0,100)):
    image_list = []

    filenames = sorted(os.listdir(directory))[min(image_range):max(image_range)+1]
    
    for filename in filenames:
        if filename.endswith(".jpg"):
            file_path = os.path.join(directory, filename)
            
            img = Image.open(file_path)
            img_resized = img.resize(target_size)
            img_array = np.array(img_resized)
            
            image_list.append(img_array)
    
    return np.array(image_list)

test_cat_images = process_images("datasets/train/cats")
test_dog_images = process_images("datasets/train/dogs")
tuning_cat_images = process_images("datasets/train/cats",image_range=(101,200))
tuning_dog_images = process_images("datasets/train/dogs",image_range=(101,200))

def join_cats_and_dogs(cat_images,dog_images):
    cat_set_X = preprocess_dataset(cat_images)
    dog_set_X = preprocess_dataset(dog_images)
    m_cat_set = cat_set_X.shape[1]
    cat_set_Y = np.ones((1, m_cat_set))
    m_dog_set = dog_set_X.shape[1]
    dog_set_Y = np.zeros((1, m_dog_set))

    set_X = np.concatenate((cat_set_X, dog_set_X), axis=1)
    set_Y = np.concatenate((cat_set_Y, dog_set_Y), axis=1)

    np.random.seed(1)
    shuffle_indices = np.random.permutation(set_X.shape[1])
    final_set_X = set_X[:, shuffle_indices]
    final_set_Y = set_Y[:, shuffle_indices]

    return final_set_X,final_set_Y

final_test_X, final_test_Y = join_cats_and_dogs(test_cat_images,test_dog_images)
final_tuning_X, final_tuning_Y = join_cats_and_dogs(tuning_cat_images,tuning_dog_images)

#### a) Use the a model (shallow fully connected network) and datasets (a complex classification problem that you can choose on your own) and test the following optimizers in the best configuration that you have obtained from Assignement 4.

We will use the same architecture for this network as the one from the last assignments given by the next formula 

In [12]:
# layers_dims = [X.shape[0], 20, 3, 1]

layers_dims = [train_set_x.shape[0], 20, 3, 1]

### 1.Gradient Descent

In [33]:
model(train_set_x, train_set_y, layers_dims, "gd", learning_rate = 0.005, num_epochs = 20000)

NameError: name 'initialize_parameters' is not defined

### 2. Momentum

### 3. Adam
 
### Show the effects of the optimizer on the final accuracy and validation loss (plot the training and validation loss)
 
### b) Test the three models using the two basic learning rate schedulers
 
### 1. Expontenial LR Decay (provided in the notebook)
 
### 2. Step LR Decay